In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import pandas as pd
import os
import json 
import numpy as np
import ast

In [10]:
os.chdir("/content/drive/MyDrive/MovieRSystem")

In this notebook, the Train and Test Split may take long!

## 1. Clean data

In [11]:
metadata = pd.read_csv("movies_metadata.csv")

In [13]:
ratings = pd.read_csv("ratings_small.csv")
metadata = pd.read_csv("movies_metadata.csv")
credits = pd.read_csv("credits.csv")
keywords = pd.read_csv("keywords.csv")

In [14]:
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [15]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

### ID

There are some records which have the `id` columns in wrong format

In [16]:
metadata[metadata['id']=="1997-08-20"]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
def isInteger(number):
    try:
        a = int(number)
    except:
        return False
    return True

In [18]:
metadata['isInteger'] = metadata['id'].apply(isInteger)
metadata[metadata['isInteger'] == False]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,isInteger
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [19]:
metadata.drop([29503, 35587, 19730], inplace=True)

In [20]:
metadata['id'] = metadata['id'].apply(lambda x: int(x))

ID should not be duplicated by records but there are some duplicated appear here.

This one appear in `metadata` dataframe

In [21]:
metadata['id'].value_counts()

141971    3
5511      2
132641    2
10991     2
168538    2
         ..
55135     1
15877     1
72272     1
1549      1
461257    1
Name: id, Length: 45433, dtype: int64

In [22]:
metadata.drop_duplicates(subset=['id'], inplace=True)

ID duplicated appear in `credits` dataframe

In [23]:
credits['id'].value_counts()

141971    3
298721    2
9755      2
10991     2
99080     2
         ..
55135     1
15877     1
72272     1
1549      1
461257    1
Name: id, Length: 45432, dtype: int64

In [24]:
credits.drop_duplicates(subset=['id'], inplace=True)

ID duplicated appear in `keywords` dataframe

In [25]:
keywords['id'].value_counts()

141971    3
159849    3
105045    2
255396    2
85729     2
         ..
55135     1
15877     1
72272     1
1549      1
461257    1
Name: id, Length: 45432, dtype: int64

In [26]:
keywords.drop_duplicates(subset=['id'], inplace=True)

## 2. Merge Datasets

Currently working on:

`genres, crew, cast, keywords, popularity, vote_average, language, runtime, vote_count, production_companies, production_countries, overview`

Merge dataset to make a full metadata of each movie. 

This is for later profile buildings.

In [27]:
dataset = metadata.merge(credits, how = 'inner', on = "id")
dataset = dataset.merge(keywords, how = 'inner', on = "id")

In [28]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 45431
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45432 non-null  object 
 1   belongs_to_collection  4488 non-null   object 
 2   budget                 45432 non-null  object 
 3   genres                 45432 non-null  object 
 4   homepage               7774 non-null   object 
 5   id                     45432 non-null  int64  
 6   imdb_id                45415 non-null  object 
 7   original_language      45421 non-null  object 
 8   original_title         45432 non-null  object 
 9   overview               44478 non-null  object 
 10  popularity             45429 non-null  object 
 11  poster_path            45046 non-null  object 
 12  production_companies   45429 non-null  object 
 13  production_countries   45429 non-null  object 
 14  release_date           45345 non-null  object 
 15  re

Get all the columns we needed.

In [29]:
columns = ['genres', 'budget', 'id', 'original_language', 'overview', 'production_companies', \
          'production_countries', 'runtime', 'revenue', 'vote_average', 'vote_count', 'cast', \
          'crew', 'keywords', 'title']

In [30]:
final_dataset = dataset[columns]
final_dataset.head()

,genres,budget,id,original_language,overview,production_companies,production_countries,runtime,revenue,vote_average,vote_count,cast,crew,keywords,title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",30000000,862,en,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",81.0,373554033.0,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",65000000,8844,en,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",104.0,262797249.0,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0,15602,en,A family wedding reignites the ancient feud be...,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",101.0,0.0,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392...",Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",127.0,81452156.0,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...",Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",0,11862,en,Just when George Banks has recovered from his ...,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",106.0,76578911.0,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...",Father of the Bride Part II


## 3. Fill Null Value

There are fields that contain null value in their records.

In [31]:
final_dataset.isnull().sum()

genres                    0
budget                    0
id                        0
original_language        11
overview                954
production_companies      3
production_countries      3
runtime                 260
revenue                   3
vote_average              3
vote_count                3
cast                      0
crew                      0
keywords                  0
title                     3
dtype: int64

### Runtime

In [32]:
final_dataset['runtime'].isnull().sum()

260

For `runtime` fill with mean of all movies' `runtime`

In [33]:
final_dataset['runtime'] = final_dataset['runtime'].fillna(final_dataset['runtime'].mean())

Fill `overview` with "" 

In [34]:
final_dataset['overview'] = final_dataset['overview'].fillna("")

## 4. Intersection

The `metadata` dataset and `ratings` dataset seems doesn't match each others by `movieId` column, so we will test the join operation to find the intersection.

Join to remove `id` field that not appear in others

Drop all na field in dataset first.

In [35]:
final_dataset = final_dataset.dropna(axis = 0).reset_index(drop = True)

Find `id` appear in both dataset

In [36]:
movie_dataset = set(final_dataset['id'])
movie_rating = set(ratings['movieId'])
intersected = movie_dataset.intersection(movie_rating)

Remove `id` that not appear in others

In [37]:
final_dataset = final_dataset[final_dataset['id'].isin(intersected)]
ratings = ratings[ratings['movieId'].isin(intersected)]

## 5. Composite Attribute Fields

Composite Fields contain raw information so we will extract the needed field from it

In [38]:
columns = ['genres', 'production_companies','production_countries', 'cast', 'crew', 'keywords']

In [39]:
for column in columns:
    final_dataset[column] = final_dataset[column].apply(ast.literal_eval)

In [40]:
final_dataset['genres'] = final_dataset['genres'].apply(lambda x: [y['name'] for y in x])

In [41]:
final_dataset['production_companies'] = final_dataset['production_companies'].apply(lambda x: [y['name'] for y in x])

In [42]:
final_dataset['production_countries'] = final_dataset['production_countries'].apply(lambda x: [y['iso_3166_1'] for y in x])

In [43]:
final_dataset['cast'] = final_dataset['cast'].apply(lambda x: [y['name'] for y in x])

In [44]:
final_dataset['crew'] = final_dataset['crew'].apply(lambda x: [y['name'] for y in x])

In [45]:
final_dataset['keywords'] = final_dataset['keywords'].apply(lambda x: [y['name'] for y in x])

The dictionary store `id` to `title` of movies

In [46]:
idToTitle = dict(zip(final_dataset['id'], final_dataset['title']))

## 6. Train and Test Split

Split the `ratings` in to `train` and `test` datasets

In [47]:
ratings

,userId,movieId,rating,timestamp
10,1,1371,2.5,1260759135
11,1,1405,1.0,1260759203
13,1,2105,4.0,1260759139
15,1,2193,2.0,1260759198
16,1,2294,2.0,1260759108
...,...,...,...,...
99983,671,4995,4.0,1064891537
99992,671,5816,4.0,1065111963
99993,671,5902,3.5,1064245507
99996,671,5991,4.5,1064245387


In [48]:
ratings['flag'] = 1

In [49]:
def split(x):
    # x is Series type
    x[int(x.size * 0.7) : None] = 0
    return pd.Series(np.random.permutation(x))

This may take long since we work with a big ratings set.

In [50]:
ratings['flag'] = ratings.groupby("userId")[['flag']].transform(split)

In [51]:
ratings

,userId,movieId,rating,timestamp,flag
10,1,1371,2.5,1260759135,1
11,1,1405,1.0,1260759203,0
13,1,2105,4.0,1260759139,0
15,1,2193,2.0,1260759198,1
16,1,2294,2.0,1260759108,1
...,...,...,...,...,...
99983,671,4995,4.0,1064891537,1
99992,671,5816,4.0,1065111963,1
99993,671,5902,3.5,1064245507,1
99996,671,5991,4.5,1064245387,1


In [52]:
ratings_train = ratings[ratings['flag'] == 1].drop(['flag', 'timestamp'], axis = 1)
ratings_test = ratings[ratings['flag'] == 0].drop(['flag', 'timestamp'], axis = 1)

## 7. Save

In [53]:
!mkdir CleanedData

In [54]:
final_dataset.to_csv("CleanedData/dataset.csv", index = False)

In [55]:
ratings_train.to_csv("CleanedData/ratings_train.csv", index = False)

In [56]:
ratings_test.to_csv("CleanedData/ratings_test.csv", index = False)

In [57]:
!mkdir ModelStorage

In [58]:
with open("ModelStorage/idToTitle.json", "w+") as file:
    file.write(json.dumps(idToTitle))